In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Set up paths
PYTHONPATH = "/global/homes/b/bpb/repos/envnet"
os.environ['PYTHONPATH'] = PYTHONPATH

# Import ENVnet
import sys
sys.path.insert(0, PYTHONPATH)
from envnet.build import quick_envnet
from envnet.config.build_config import BuildConfig
from envnet.build import ENVnetBuilder

output_dir = Path(os.path.join(PYTHONPATH, "results", "temp_data"))
output_dir.mkdir(parents=True, exist_ok=True)

# Create config
config = BuildConfig()
# config.remblink_cutoff = 0.05
# config.network_max_mz_difference = 5000.0
config.min_cluster_size = 3
print(config.max_rt)

70.0


In [ ]:
/global/homes/b/bpb/repos/envnet/scripts/my_files_test.csv

In [4]:
import pandas as pd
import os
filename = '/global/homes/b/bpb/repos/envnet/scripts/my_files.csv'
df = pd.read_csv(filename)
# replace all text having 'carbon_network/raw_data' with 'envnet_build_files' in the entire dataframe
for c in df.columns:
    df[c] = df[c].str.replace('carbon_network/raw_data', 'envnet_build_files', regex=False)
df = df[~df['parquet'].str.contains('MSV000081030')]
idx = df['parquet'].str.contains('_deconvoluted.parquet')
df.loc[~idx,'parquet'] = df.loc[~idx,'parquet'].str.replace('.parquet','_deconvoluted.parquet', regex=False)

print(df.shape)
df.to_csv('/global/homes/b/bpb/repos/envnet/scripts/build_files.csv', index=False)
for f in df.head()['parquet'].tolist():
    print(f)
    print(os.path.exists(f))

(1389, 3)
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230223_EB_MdR_101544-059_SynDAC_20230223_QE144_C18-EP_USDAY72350_NEG_MS2_29_RS-HA-NA_1__18_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20221219_EB_MdR_101544-059_PyrToDAC_20221219_EXP120A_C18-EP_USDAY63672_NEG_MS2_8_FAIII-1mg-NA_1__48_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_13_SRFA3-H2O-fil-NA-NA_1__18_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_14_SRFA3-H2O-unfil-NA-NA_1__36_deconvoluted.parquet
True
/global/cfs/cdirs/metatlas/projects/envnet_build_files/metatlas/20230113_EB_MdR_101544-059_FRCgwater_20230105_EXP120A_C18-EP_USDAY72349_NEG_MS2_15_SRFA3-MeOH-fil-NA-NA_1__12_deconvoluted.parquet
True


In [4]:
node_data.columns

Index(['original_index', 'temp_index', 'rt', 'count', 'precursor_mz',
       'sum_frag_intensity', 'max_frag_intensity', 'obs',
       'isolated_precursor_mz', 'filename', 'basename',
       'coisolated_precursor_count', 'coisolated_precursor_mz_list',
       'assumed_adduct', 'predicted_formula', 'estimated_fdr',
       'predicted_mass', 'mass_error', 'num_datapoints', 'peak_area',
       'peak_height', 'mz_centroid', 'rt_peak', 'score', 'matches',
       'library_formula', 'library_precursor_mz', 'inchi_key', 'compound_name',
       'smiles', 'match_type', 'duplicate_cluster_index', 'num_duplicates',
       'dbe', 'dbe_ai', 'dbe_ai_mod', 'ai_mod', 'ai', 'nosc', 'h_to_c',
       'o_to_c', 'n_to_c', 'p_to_c', 'n_to_p', 'c', 'h', 'o', 'n', 's', 'p',
       'has_library_match', 'is_singleton'],
      dtype='object')

In [5]:
import pandas as pd
filename = '/global/u2/b/bpb/repos/envnet/results/full_build_20250908_181404/envnet_node_data.csv'
node_data = pd.read_csv(filename)
node_data['inchi_key'].nunique()

/tmp/ipykernel_53833/3805686495.py:3: DtypeWarning: Columns (25,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  node_data = pd.read_csv(filename)


709

In [ ]:
# use recursive find to get all parquet files
# use linux find since it is much faster than glob
cmd = "find /global/cfs/cdirs/metatlas/projects/envnet_build_files/ -name '*_deconvoluted.parquet'"
# run command
import subprocess
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
files = result.stdout.splitlines()
len(files)

In [ ]:
# # remove MSV000081030 files.  These seem to be not environmental
files = [f for f in files if "MSV000081030" not in f] # MetaboLights MTBLS144 - GNPS Dissolved organic matter produced by Thalassiosira pseudonana

print(len(files))
# files = [f for f in files if "MSV000088008" not in f]

# # files = [f for f in files if "DOM_Interlab-LCMS_Lab024" not in f]
# len(files)

In [ ]:
import os
for f in files:
    mzml_filename = f.replace("_deconvoluted.parquet", ".mzML")
    h5_filename = f.replace("_deconvoluted.parquet", ".h5")
    if not os.path.exists(mzml_filename):
        print(f)
    if not os.path.exists(h5_filename):
        print(f)

In [ ]:
# import shutil
# path_to_replace = '/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data'
# path_replace_with = '/global/cfs/cdirs/metatlas/projects/envnet_build_files'
# for f in files:
#     mzml_filename = f.replace("_deconvoluted.parquet", ".mzML")
#     h5_filename = f.replace("_deconvoluted.parquet", ".h5")
#     new_mzml_filename = mzml_filename.replace(path_to_replace, path_replace_with)
#     new_h5_filename = h5_filename.replace(path_to_replace, path_replace_with)
#     # copy the mzml and h5 file from the original path to the new path
#     # make the sub directories
#     os.makedirs(os.path.dirname(new_mzml_filename), exist_ok=True)
#     shutil.copy(mzml_filename, new_mzml_filename)
#     shutil.copy(h5_filename, new_h5_filename)


In [ ]:
df = []
for parquet_file in files:
    temp_df = pd.read_parquet(parquet_file)
    # max_rt = temp_df['rt'].max()
    # filtered_rt_max = max_rt*0.7
    # temp_df = temp_df[temp_df['rt']<filtered_rt_max]

    # temp_df = temp_df[temp_df['rt']<config.max_rt]
    df.append(temp_df)

df = pd.concat(df, ignore_index=True)
# df = df[df['rt']<config.max_rt]
# df = df[df['rt']>config.min_rt]
df.reset_index(drop=True, inplace=True)

In [ ]:
# # parquet_file = '/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/metatlas/20220707_JGI_SB_503799_Permafrost_pilot_QE-HF_C18_USDAY63663_NEG_MSMS_10_55M-Subsection-MeOH_1_Rg80to1200-CE102040-soil-S1_Run26_deconvoluted.parquet'
# # print(f"Loading: {parquet_file}")
# # df = pd.read_parquet(parquet_file)


# import glob
# files1 = glob.glob('/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/metatlas/20220707_JGI_SB_503799_*_deconvoluted.parquet')
# files2 = glob.glob('/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/massive/v06/MSV000092520/peak/*_deconvoluted.parquet')
# files3 = glob.glob('/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data/metatlas/20240112_JGI_MdR_109570-002_OMTSoil50g_Pilot_QEHF_C18_USDAY86082_NEG_MS2_1_soil-*_deconvoluted.parquet')
# files = files1 + files2 + files3
# df = []

# for parquet_file in files:
#     print(f"Loading: {parquet_file}")
#     temp_df = pd.read_parquet(parquet_file)
#     df.append(temp_df)

# df = pd.concat(df, ignore_index=True)
# df.reset_index(drop=True, inplace=True)

In [ ]:



print(f"Total spectra in file: {len(df)}")
print(f"Columns: {list(df.columns)}")
print(f"Precursor m/z range: {df['precursor_mz'].min():.3f} - {df['precursor_mz'].max():.3f}")

df.rename(columns={
    'mdm_mz_vals': 'deconvoluted_spectrum_mz_vals',
    'mdm_i_vals': 'deconvoluted_spectrum_intensity_vals',
    'original_mz_vals':'original_spectrum_mz_vals',
    'original_i_vals':'original_spectrum_intensity_vals'
}, inplace=True)

df['basename'] = df['filename'].apply(lambda x: os.path.basename(x))
df.head()


In [ ]:

# # 2. Select strategic spectra for testing
# # Goal: Pick some that should cluster together and some that shouldn't

# # Strategy 1: Pick spectra with similar m/z (should connect if similar fragmentation)
# similar_mz_group = df[
#     (df['precursor_mz'] >= 200) & (df['precursor_mz'] <= 210)
# ].head(5)

# # Strategy 2: Pick spectra with very different m/z (should NOT connect)  
# different_mz_1 = df[
#     (df['precursor_mz'] >= 300) & (df['precursor_mz'] <= 310)
# ].head(3)

# different_mz_2 = df[
#     (df['precursor_mz'] >= 500) & (df['precursor_mz'] <= 510)
# ].head(3)

# # Strategy 3: Pick a few random ones
# random_spectra = df.sample(n=min(5, len(df)), random_state=42)

# # Combine all test spectra
# test_spectra = pd.concat([
#     similar_mz_group,
#     different_mz_1, 
#     different_mz_2,
#     random_spectra
# ]).head(15).reset_index(drop=True)  # Limit to 15 spectra



# print(f"\nSelected {len(test_spectra)} test spectra:")
# print("Similar m/z group (should potentially connect):")
# for _, row in similar_mz_group.iterrows():
#     print(f"  m/z: {row['precursor_mz']:.3f}")

# print("Different m/z groups (should be separate):")
# for _, row in different_mz_1.iterrows():
#     print(f"  m/z: {row['precursor_mz']:.3f}")
# for _, row in different_mz_2.iterrows():
#     print(f"  m/z: {row['precursor_mz']:.3f}")





# this is from the log of running the full build from what is in the sbatch file

```bash
Found 6312 unique precursor m/z groups
Counter: 0, Processing 59886 entries for m/z range 352.9960 to 353.1623, Found 664 unique spectra
Counter: 1, Processing 56025 entries for m/z range 324.9888 to 325.1676, Found 510 unique spectra
Counter: 2, Processing 55959 entries for m/z range 381.0351 to 381.1588, Found 551 unique spectra
Counter: 3, Processing 55516 entries for m/z range 367.0203 to 367.1440, Found 615 unique spectra
Counter: 4, Processing 54880 entries for m/z range 339.0132 to 339.1479, Found 472 unique spectra
Counter: 5, Processing 50100 entries for m/z range 369.0166 to 369.1595, Found 623 unique spectra
Counter: 6, Processing 49832 entries for m/z range 395.0509 to 395.1767, Found 633 unique spectra
Counter: 7, Processing 49046 entries for m/z range 379.0115 to 379.1803, Found 543 unique spectra
Counter: 8, Processing 44056 entries for m/z range 365.0063 to 365.1624, Found 450 unique spectra
Counter: 9, Processing 43186 entries for m/z range 311.0153 to 311.2248, Found 644 unique spectra
Counter: 10, Processing 40995 entries for m/z range 409.0668 to 409.1936, Found 616 unique spectra
Counter: 11, Processing 39164 entries for m/z range 337.0303 to 337.1704, Found 391 unique spectra
Counter: 12, Processing 37273 entries for m/z range 423.0173 to 423.1874, Found 651 unique spectra
Counter: 13, Processing 34520 entries for m/z range 341.0252 to 341.1286, Found 439 unique spectra
```



In [ ]:
test_spectra = df.copy()

builder = ENVnetBuilder(config=config, verbose=True)
# IMPORTANT: Preprocess the spectra
print("Preprocessing spectra...")
test_spectra = builder.workflows.data_loader._preprocess_spectra(test_spectra)

# Mock the data loading by directly setting the spectra
# builder.workflows.data_loader.all_spectra = test_spectra.copy()

# Run library matching
print("Running library matching...")
# deconv_matches = builder.workflows.library_matcher.score_all_spectra(test_spectra, scoring_type='deconvoluted')
# orig_matches = builder.workflows.library_matcher.score_all_spectra(test_spectra, scoring_type='original')
# deconv_matches = pd.DataFrame(columns=['spectrum_id', 'library_id', 'score', 'scoring_type'])
# orig_matches = pd.DataFrame(columns=['spectrum_id', 'library_id', 'score', 'scoring_type'])
# Run clustering
print("Running clustering...")
clustered_spectra = builder.workflows.clusterer.cluster_duplicate_spectra(test_spectra)

## the cluster_duplicate_spectra is dead on identical to the full workflow


In [ ]:
# export clustered spectra to parquet
filename = os.path.join( config.metadata_folder, "toy_network_clustered_spectra-freshconversion.parquet")
clustered_spectra.to_parquet(filename, index=False)

In [6]:
filename = os.path.join( config.metadata_folder, "toy_network_clustered_spectra-freshconversion.parquet")
clustered_spectra = pd.read_parquet(filename)

In [7]:

config.min_cluster_size = 4
builder = ENVnetBuilder(config=config, verbose=True)


# Eliminate redundant spectra
print("Eliminating redundant spectra...")
node_data = builder.workflows.clusterer.eliminate_redundant_spectra(clustered_spectra)#.sample(frac=0.1))
    # clustered_spectra, deconv_matches, orig_matches
# )
# This will be slightly different, but only in which spectra result in being chosen.  Below, they are chosen based on library matches, but here only on peak area
# this is what the full workflow logged
#    Identified 739 unique spectral clusters

# 4. Eliminating redundant spectra...
# Final dataset: 24713 unique spectra
#    Reduced to 24713 unique representative spectra

# 5. Building molecular network...
# Building network from 24713 spectra...
    # min_cluster_size: int = 5

# 2: Final dataset: 123191 unique spectra
# 5: Final dataset: 14696 unique spectra


ENVnet Builder initialized
Eliminating redundant spectra...
Info: No deconvoluted library matches provided.
Info: No original library matches provided.
Final dataset: 22128 unique spectra


In [8]:
# for each attr in config if int or float, print the attr and its value
for attr in dir(config):
    if not attr.startswith("_"):
        value = getattr(config, attr)
        if isinstance(value, (int, float)):
            print(f"{attr}: {value}")

bin_width: 0.001
chunk_size: 2000
intensity_power: 0.5
max_rt: 70.0
min_cluster_size: 4
min_matches: 3
min_rt: 0.5
min_score: 0.9
mz_tol: 0.002
network_max_mz_difference: 1000.0
override_matches: 20
remblink_cutoff: 0.05
verbose: True


In [9]:
# Build network
print("Building network...")
# config.remblink_cutoff = 0.05
# config.network_max_mz_difference = 5000.0

builder = ENVnetBuilder(config=config, verbose=True)

network = builder.workflows.network_builder.build_remblink_network(node_data)


# drop singletons from networkx network
components = nx.connected_components(network)

# # 2. Create a list of actual subgraph objects for components that meet the size criteria
large_subgraphs = [network.subgraph(c).copy() for c in components if len(c) > 10]

# 3. Compose the large subgraphs into the final network
if large_subgraphs:
    network = nx.compose_all(large_subgraphs)

output_network = os.path.join(output_dir, "toy_network-no-MSV000081030-0p05-0p9-4-again.graphml")
nx.write_graphml(network, output_network)

Building network...
ENVnet Builder initialized
Building network from 22128 spectra...
Processing 12 chunks...
Processing chunk 1/12
Using model: /global/u2/b/bpb/repos/envnet/envnet/data/mdm_negative_random_forest.joblib


/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing chunk 2/12
Using model: /global/u2/b/bpb/repos/envnet/envnet/data/mdm_negative_random_forest.joblib


/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing chunk 3/12
Using model: /global/u2/b/bpb/repos/envnet/envnet/data/mdm_negative_random_forest.joblib


...

Processing chunk 11/12
Using model: /global/u2/b/bpb/repos/envnet/envnet/data/mdm_negative_random_forest.joblib


/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing chunk 12/12
Using model: /global/u2/b/bpb/repos/envnet/envnet/data/mdm_negative_random_forest.joblib


/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/global/common/software/m2650/msbuddy/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Total edges before filtering: 810775
Edges after score cutoff (0.05): 810775
Edges after m/z cutoff (1000.0): 810775
Network nodes: 22128
Network edges: 397235
Connected components: 6302
Nodes from original data in network: 22128/22128


In [ ]:
from envnet.build.mgf_tools import MGFGenerator
print("\nGenerating MGF files...")
mgf_generator = MGFGenerator(config)
created_mgf_files = mgf_generator.create_mgf_files(
    node_data=node_data,
    network=network,
    output_dir=str(output_dir)
)

print("MGF generation complete. Files created:")
for spec_type, file_path in created_mgf_files.items():
    print(f"  - {spec_type.capitalize()}: {file_path}")

# here is the log from full build

```bash
Total edges before filtering: 195381
Edges after score cutoff (0.05): 195381
Edges after m/z cutoff (1000.0): 195381
Network nodes: 24713
Network edges: 86583
Connected components: 13731
Nodes from original data in network: 24713/24713
Network: 24713 nodes, 86583 edges
Connected components: 13731
```


In [ ]:
# plt.figure(figsize=(20, 20))
# if network.number_of_nodes() > 0:
#     # pos = nx.spring_layout(network, k=2, iterations=500)
#     pos = nx.nx_agraph.graphviz_layout(network, prog='neato')

#     # Draw nodes colored by precursor m/z
#     node_colors = [network.nodes[node].get('precursor_mz', 0) for node in network.nodes()]
    
#     nodes = nx.draw_networkx_nodes(network, pos, 
#                             node_color=node_colors, 
#                             node_size=10,
#                             cmap='viridis',
#                             alpha=0.8,
#                             vmin=150,
#                             vmax=500)
    
#     nx.draw_networkx_edges(network, pos, alpha=0.25, width=1)
    
#     # Add labels with m/z values
#     # labels = {node: f"{network.nodes[node].get('precursor_mz', 0):.4f}" 
#     #             for node in network.nodes()}
#     # nx.draw_networkx_labels(network, pos, labels, font_size=8)
    
#     plt.colorbar(nodes, label='Precursor m/z')

#     plt.title(f"ENVnet Test Network\n{network.number_of_nodes()} nodes, {network.number_of_edges()} edges")
    
# else:
#     plt.text(0.5, 0.5, "No network generated", 
#             horizontalalignment='center', verticalalignment='center')
#     plt.title("ENVnet Test - No Network")

# plt.axis('off')
# plt.tight_layout()
# plt.show()

